In [37]:
from transformers import AutoTokenizer
from collections import defaultdict

In [38]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [39]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [40]:
def norm_tokenizer(text):
    normalizer = tokenizer.backend_tokenizer.normalizer
    pretokenizer = tokenizer.backend_tokenizer.pre_tokenizer
    return pretokenizer.pre_tokenize_str(normalizer.normalize_str(text))

In [41]:
pretokenizer.pre_tokenize_str(corpus[0])

[('This', (0, 4)),
 ('is', (5, 7)),
 ('the', (8, 11)),
 ('Hugging', (12, 19)),
 ('Face', (20, 24)),
 ('Course', (25, 31)),
 ('.', (31, 32))]

In [42]:
norm_tokenizer(corpus[0])

[('this', (0, 4)),
 ('is', (5, 7)),
 ('the', (8, 11)),
 ('hugging', (12, 19)),
 ('face', (20, 24)),
 ('course', (25, 31)),
 ('.', (31, 32))]

In [43]:
word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = norm_tokenizer(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'this': 3,
             'is': 2,
             'the': 1,
             'hugging': 1,
             'face': 1,
             'course': 1,
             '.': 4,
             'chapter': 1,
             'about': 1,
             'tokenization': 1,
             'section': 1,
             'shows': 1,
             'several': 1,
             'tokenizer': 1,
             'algorithms': 1,
             'hopefully': 1,
             ',': 1,
             'you': 1,
             'will': 1,
             'be': 1,
             'able': 1,
             'to': 1,
             'understand': 1,
             'how': 1,
             'they': 1,
             'are': 1,
             'trained': 1,
             'and': 1,
             'generate': 1,
             'tokens': 1})

In [44]:
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

print(alphabet)

['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'a', 'b', 'c', 'f', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y']


In [45]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

In [46]:
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}

In [47]:
splits

{'this': ['t', '##h', '##i', '##s'],
 'is': ['i', '##s'],
 'the': ['t', '##h', '##e'],
 'hugging': ['h', '##u', '##g', '##g', '##i', '##n', '##g'],
 'face': ['f', '##a', '##c', '##e'],
 'course': ['c', '##o', '##u', '##r', '##s', '##e'],
 '.': ['.'],
 'chapter': ['c', '##h', '##a', '##p', '##t', '##e', '##r'],
 'about': ['a', '##b', '##o', '##u', '##t'],
 'tokenization': ['t',
  '##o',
  '##k',
  '##e',
  '##n',
  '##i',
  '##z',
  '##a',
  '##t',
  '##i',
  '##o',
  '##n'],
 'section': ['s', '##e', '##c', '##t', '##i', '##o', '##n'],
 'shows': ['s', '##h', '##o', '##w', '##s'],
 'several': ['s', '##e', '##v', '##e', '##r', '##a', '##l'],
 'tokenizer': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##e', '##r'],
 'algorithms': ['a',
  '##l',
  '##g',
  '##o',
  '##r',
  '##i',
  '##t',
  '##h',
  '##m',
  '##s'],
 'hopefully': ['h', '##o', '##p', '##e', '##f', '##u', '##l', '##l', '##y'],
 ',': [','],
 'you': ['y', '##o', '##u'],
 'will': ['w', '##i', '##l', '##l'],
 'be': ['b', '##e

In [48]:
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [49]:
pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

('t', '##h'): 0.0625
('##h', '##i'): 0.03409090909090909
('##i', '##s'): 0.02727272727272727
('i', '##s'): 0.1
('##h', '##e'): 0.011904761904761904
('h', '##u'): 0.06666666666666667


In [50]:
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('a', '##b') 0.2


In [51]:
vocab.append("ab")

In [52]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [53]:
splits = merge_pair("a", "##b", splits)
splits["about"]

['ab', '##o', '##u', '##t']

In [54]:
splits

{'this': ['t', '##h', '##i', '##s'],
 'is': ['i', '##s'],
 'the': ['t', '##h', '##e'],
 'hugging': ['h', '##u', '##g', '##g', '##i', '##n', '##g'],
 'face': ['f', '##a', '##c', '##e'],
 'course': ['c', '##o', '##u', '##r', '##s', '##e'],
 '.': ['.'],
 'chapter': ['c', '##h', '##a', '##p', '##t', '##e', '##r'],
 'about': ['ab', '##o', '##u', '##t'],
 'tokenization': ['t',
  '##o',
  '##k',
  '##e',
  '##n',
  '##i',
  '##z',
  '##a',
  '##t',
  '##i',
  '##o',
  '##n'],
 'section': ['s', '##e', '##c', '##t', '##i', '##o', '##n'],
 'shows': ['s', '##h', '##o', '##w', '##s'],
 'several': ['s', '##e', '##v', '##e', '##r', '##a', '##l'],
 'tokenizer': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##e', '##r'],
 'algorithms': ['a',
  '##l',
  '##g',
  '##o',
  '##r',
  '##i',
  '##t',
  '##h',
  '##m',
  '##s'],
 'hopefully': ['h', '##o', '##p', '##e', '##f', '##u', '##l', '##l', '##y'],
 ',': [','],
 'you': ['y', '##o', '##u'],
 'will': ['w', '##i', '##l', '##l'],
 'be': ['b', '##e'],
 '

In [55]:
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

In [56]:
print(vocab)


['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'a', 'b', 'c', 'f', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y', 'ab', '##fu', 'fa', 'fac', '##ct', '##ful', '##full', '##fully', '##hm', '##thm', 'is', '##thms', '##pt', '##apt', '##hapt', 'chapt', '##za', '##zat', 'abl', '##izat', '##izati', '##cti', '##iz', '##ithms', 'wi', 'wil', 'will', 'al']


In [57]:
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

In [59]:
print(encode_word("hugging"))
print(encode_word("hOgging"))

['h', '##u', '##g', '##g', '##i', '##n', '##g']
['[UNK]']


In [60]:
def tokenize(text):
    pre_tokenize_result = norm_tokenizer(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

In [61]:
tokenize("This is the Hugging Face course!")

['t',
 '##h',
 '##i',
 '##s',
 'is',
 't',
 '##h',
 '##e',
 'h',
 '##u',
 '##g',
 '##g',
 '##i',
 '##n',
 '##g',
 'fac',
 '##e',
 'c',
 '##o',
 '##u',
 '##r',
 '##s',
 '##e',
 '[UNK]']

In [63]:
set(vocab)

{'##a',
 '##apt',
 '##b',
 '##c',
 '##ct',
 '##cti',
 '##d',
 '##e',
 '##f',
 '##fu',
 '##ful',
 '##full',
 '##fully',
 '##g',
 '##h',
 '##hapt',
 '##hm',
 '##i',
 '##ithms',
 '##iz',
 '##izat',
 '##izati',
 '##k',
 '##l',
 '##m',
 '##n',
 '##o',
 '##p',
 '##pt',
 '##r',
 '##s',
 '##t',
 '##thm',
 '##thms',
 '##u',
 '##v',
 '##w',
 '##y',
 '##z',
 '##za',
 '##zat',
 ',',
 '.',
 '[CLS]',
 '[MASK]',
 '[PAD]',
 '[SEP]',
 '[UNK]',
 'a',
 'ab',
 'abl',
 'al',
 'b',
 'c',
 'chapt',
 'f',
 'fa',
 'fac',
 'g',
 'h',
 'i',
 'is',
 's',
 't',
 'u',
 'w',
 'wi',
 'wil',
 'will',
 'y'}